In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

1. Проведите небольшое исследование алгоритма градиентного спуска на данных из урока. Оцените влияние значений скорости обучения (eta) и количества итераций на ошибку алгоритма. Как связаны эти два гиперпараметра между собой? Подберите скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК. Как можно ускорить процесс вычисления весов?



In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # для умножения на intercept
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2]]).T # стаж репетитора
X.shape

(10, 2)

In [3]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [4]:
y = [45, 55, 50, 59, 65, 35, 75, 80, 50, 60]

Если скорость обучения мала, то обучение более надежно, но это займет много времени, потому что шаги к минимуму функции потерь незначительны. Для того, чтобы достигнуть целевого показателя 
МНК: array([47.23214286,  3.91071429]) 45.937499999999986
при eta = 1e-3 потребуется около 12001 итераций

In [5]:
n = X.shape[0]
eta = 1e-3
iterations = 12001
w = np.array([1, 0.5])

for i in range(iterations):
    y_pred = np.dot(w, X.T)
    err = calc_mse(y, y_pred)
    for j in range(w.shape[0]):
        w[j] -= eta * (1/n * 2 * np.sum(X[:,j] * (y_pred - y)))
    if i % 1000 == 0:
        print(i, w, err) # МНК: array([47.23214286,  3.91071429]) 45.937499999999986

0 [1.1102 0.84  ] 3173.15
1000 [31.66648131  6.78503276] 176.46806566379882
2000 [41.73215361  4.92632937] 62.2342399245102
3000 [45.28877016  4.26957292] 47.97214782992787
4000 [46.54546925  4.03751382] 46.19152576288303
5000 [46.98951279  3.95551778] 45.96921511416331
6000 [47.14641166  3.92654521] 45.94145963171208
7000 [47.2018505  3.916308 ] 45.93799435998288
8000 [47.22143932  3.91269078] 45.937561720839334
9000 [47.22836086  3.91141266] 45.93750770584623
10000 [47.23080652  3.91096105] 45.93750096207482
11000 [47.23167067  3.91080148] 45.93750012011503
12000 [47.23197602  3.91074509] 45.937500014996345


Если уменьшить шаг градиентного спуска, то количество этераций для досижения целевого показателя будет меньше.
eta = 1e-2 и соответсвенно скорость выполнения будет быстрее.

In [6]:
n = X.shape[0]
eta = 1e-2
iterations = 1001
w = np.array([1, 0.5])

for i in range(iterations):
    y_pred = np.dot(w, X.T)
    err = calc_mse(y, y_pred)
    for j in range(w.shape[0]):
        w[j] -= eta * (1/n * 2 * np.sum(X[:,j] * (y_pred - y)))
    if i % 100 == 0:
        print(i, w, err) # МНК: array([47.23214286,  3.91071429]) 45.937499999999986

0 [2.102 3.9  ] 3173.15
100 [31.88770806  6.74418155] 175.19445858001853
200 [41.83683774  4.90699865] 61.9177717428135
300 [45.33508261  4.26102097] 47.913169919666785
400 [46.56511152  4.03388672] 46.181755648107604
500 [46.99760587  3.95402334] 45.96769776787538
600 [47.14967657  3.92594232] 45.941233404700036
700 [47.20314662  3.91606866] 45.93796156758051
800 [47.2219474   3.91259695] 45.93755706443538
900 [47.228558    3.91137626] 45.937507054979434
1000 [47.23088237  3.91094704] 45.937500872219864


2. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее:

In [7]:
w = np.array([1, 0.5])
n = X.shape[0]
eta = 1e-2
iterations = 1001

for i in range(iterations):
    y_pred = np.dot(w, X.T)
    err = calc_mse(y, y_pred)
#    w -= (eta * (1/n * 2 * np.sum(X.T * (y_pred - y)))) # ошибка!
    w -= (eta * (1/n * 2 * np.dot(X.T, (y_pred - y))))
    if i % 100 == 0:
        print(i, w, err) # [47.23214286  3.91071429] 45.937499999999986

0 [2.102 3.9  ] 3173.15
100 [31.88770806  6.74418155] 175.19445858001842
200 [41.83683774  4.90699865] 61.9177717428135
300 [45.33508261  4.26102097] 47.913169919666785
400 [46.56511152  4.03388672] 46.181755648107604
500 [46.99760587  3.95402334] 45.96769776787538
600 [47.14967657  3.92594232] 45.941233404700036
700 [47.20314662  3.91606866] 45.93796156758051
800 [47.2219474   3.91259695] 45.93755706443538
900 [47.228558    3.91137626] 45.937507054979434
1000 [47.23088237  3.91094704] 45.937500872219864


Решение:     w -= (eta * (1/n * 2 * np.dot(X.T, (y_pred - y))))

*3. Вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию начинает изменяться ниже определенного порога. 

Сколько нужно сделать итераций, если установить допустимое отклонение mse в размере $\text{diff}=10^{-6}$, а значение $\eta=10^{-2}$?

In [8]:
w = np.array([1, 0.5])
diff = 1e-6
eta = 1e-2

In [9]:
# шаг градиентного спуска
eta = 1e-2

# критерий сходимости (разница весов, при которой алгоритм останавливается)
min_weight_dist = 1e-6

# зададим начальную разницу весов большим числом
weight_dist = np.inf

# счетчик итераций
iter_num = 0

# ход градиентного спуска
while weight_dist > min_weight_dist:
    for j in range(w.shape[0]):
        w[j] -= eta * (1/n * 2 * np.sum(X[:,j] * (y_pred - y)))
        iter_num += 1
        new_w = w - 2 * eta * (1/n * 2 * np.sum(X[:,j] * (y_pred - y)))
        weight_dist = np.linalg.norm(new_w - w, ord=2)
    if iter_num % 100 == 0:
        print(i, w, err)
    
print(f'В случае использования градиентного спуска функционал ошибки составляет {round(iter_num, 4)}')

1000 [1.0006622  0.49987772] 45.937500872219864
1000 [1.0013244  0.49975544] 45.937500872219864
1000 [1.00198659 0.49963316] 45.937500872219864
1000 [1.00264879 0.49951088] 45.937500872219864
1000 [1.00331099 0.4993886 ] 45.937500872219864
1000 [1.00397319 0.49926632] 45.937500872219864
1000 [1.00463539 0.49914404] 45.937500872219864
1000 [1.00529758 0.49902176] 45.937500872219864
1000 [1.00595978 0.49889948] 45.937500872219864
1000 [1.00662198 0.4987772 ] 45.937500872219864
1000 [1.00728418 0.49865492] 45.937500872219864
1000 [1.00794638 0.49853264] 45.937500872219864
1000 [1.00860857 0.49841036] 45.937500872219864
1000 [1.00927077 0.49828808] 45.937500872219864
1000 [1.00993297 0.4981658 ] 45.937500872219864
1000 [1.01059517 0.49804352] 45.937500872219864
1000 [1.01125736 0.49792124] 45.937500872219864
1000 [1.01191956 0.49779896] 45.937500872219864
1000 [1.01258176 0.49767668] 45.937500872219864
1000 [1.01324396 0.4975544 ] 45.937500872219864
1000 [1.01390616 0.49743212] 45.93750087

1000 [1.13707497 0.47468806] 45.937500872219864
1000 [1.13773717 0.47456578] 45.937500872219864
1000 [1.13839937 0.4744435 ] 45.937500872219864
1000 [1.13906157 0.47432122] 45.937500872219864
1000 [1.13972377 0.47419894] 45.937500872219864
1000 [1.14038596 0.47407666] 45.937500872219864
1000 [1.14104816 0.47395438] 45.937500872219864
1000 [1.14171036 0.4738321 ] 45.937500872219864
1000 [1.14237256 0.47370982] 45.937500872219864
1000 [1.14303475 0.47358754] 45.937500872219864
1000 [1.14369695 0.47346526] 45.937500872219864
1000 [1.14435915 0.47334298] 45.937500872219864
1000 [1.14502135 0.4732207 ] 45.937500872219864
1000 [1.14568355 0.47309842] 45.937500872219864
1000 [1.14634574 0.47297614] 45.937500872219864
1000 [1.14700794 0.47285386] 45.937500872219864
1000 [1.14767014 0.47273158] 45.937500872219864
1000 [1.14833234 0.4726093 ] 45.937500872219864
1000 [1.14899454 0.47248702] 45.937500872219864
1000 [1.14965673 0.47236474] 45.937500872219864
1000 [1.15031893 0.47224246] 45.93750087

1000 [1.27150116 0.44986523] 45.937500872219864
1000 [1.27216335 0.44974295] 45.937500872219864
1000 [1.27282555 0.44962067] 45.937500872219864
1000 [1.27348775 0.4494984 ] 45.937500872219864
1000 [1.27414995 0.44937612] 45.937500872219864
1000 [1.27481214 0.44925384] 45.937500872219864
1000 [1.27547434 0.44913156] 45.937500872219864
1000 [1.27613654 0.44900928] 45.937500872219864
1000 [1.27679874 0.448887  ] 45.937500872219864
1000 [1.27746094 0.44876472] 45.937500872219864
1000 [1.27812313 0.44864244] 45.937500872219864
1000 [1.27878533 0.44852016] 45.937500872219864
1000 [1.27944753 0.44839788] 45.937500872219864
1000 [1.28010973 0.4482756 ] 45.937500872219864
1000 [1.28077193 0.44815332] 45.937500872219864
1000 [1.28143412 0.44803104] 45.937500872219864
1000 [1.28209632 0.44790876] 45.937500872219864
1000 [1.28275852 0.44778648] 45.937500872219864
1000 [1.28342072 0.4476642 ] 45.937500872219864
1000 [1.28408292 0.44754192] 45.937500872219864
1000 [1.28474511 0.44741964] 45.93750087

1000 [1.41453591 0.42345277] 45.937500872219864
1000 [1.41519811 0.42333049] 45.937500872219864
1000 [1.41586031 0.42320821] 45.937500872219864
1000 [1.4165225  0.42308593] 45.937500872219864
1000 [1.4171847  0.42296365] 45.937500872219864
1000 [1.4178469  0.42284137] 45.937500872219864
1000 [1.4185091  0.42271909] 45.937500872219864
1000 [1.4191713  0.42259681] 45.937500872219864
1000 [1.41983349 0.42247453] 45.937500872219864
1000 [1.42049569 0.42235225] 45.937500872219864
1000 [1.42115789 0.42222997] 45.937500872219864
1000 [1.42182009 0.42210769] 45.937500872219864
1000 [1.42248229 0.42198541] 45.937500872219864
1000 [1.42314448 0.42186313] 45.937500872219864
1000 [1.42380668 0.42174085] 45.937500872219864
1000 [1.42446888 0.42161857] 45.937500872219864
1000 [1.42513108 0.42149629] 45.937500872219864
1000 [1.42579327 0.42137401] 45.937500872219864
1000 [1.42645547 0.42125173] 45.937500872219864
1000 [1.42711767 0.42112945] 45.937500872219864
1000 [1.42777987 0.42100717] 45.93750087

KeyboardInterrupt: 